In [1]:
import numpy as np

In [2]:
def forward_substitution(lower, b):
    n = len(lower)
    if np.iscomplexobj(lower) or np.iscomplexobj(b):
        Sol = np.empty(n,complex)
    else:
        Sol = np.empty(n,float)


    for i in range(n):
        Sol[i] = (b[i] - lower[i, :i] @ Sol[:i]) / lower[i, i]

    return Sol

def backward_substitution(upper, b):
    n = len(upper)
    if np.iscomplexobj(upper) or np.iscomplexobj(b):
        Sol = np.empty(n,complex)
    else:
        Sol = np.empty(n,float)

    for i in range(n-1, -1, -1):
        Sol[i] = (b[i] - upper[i, i+1:] @ Sol[i+1:]) / upper[i, i]

    return Sol

def Elim_Gauss(a,b):# Gaussian elimination
    N = len(b)

    for m in range(N):
        # Divide by the diagonal element
        div = a[m,m]
        a[m,:] /= div
        b[m] /= div

        # Now subtract from the lower rows
        for i in range(m+1,N):
            mult = a[i,m]
            a[i,:] -= mult*a[m,:]
            b[i] -= mult*b[m]

    return backward_substitution(a,b)

def ParcPivot(a,b):
    indzero = np.asarray(np.where(a==0))
    NumbColumn = np.zeros(len(b))
    acop = a.copy()
    bcop = b.copy()
    
    if len(indzero)!= 0:
        for n in range(len(b)):
            NumbColumn[n] = np.count_nonzero(indzero[1]==n)
    
    NumbColumn = np.argsort(NumbColumn)[::-1]
        
    maximum = np.max(np.abs(acop))
    
    for k in NumbColumn[:-1]:
        p = np.argmax(np.abs(acop[:,k]))
        if p != k:
            a[[k,p]] = a[[p,k]]
            b[[k,p]] = b[[p,k]] 
            (acop[k,:],acop[p,:]) = (acop[k,:],np.zeros(len(acop[k,:])))
        else:
            acop[k,:] = np.zeros(len(acop[k,:]))
    return

def Elim_GaussPivot(a,b):# Gaussian elimination
    N = len(b)
    
    ParcPivot(a,b)

    for m in range(N):
        # Divide by the diagonal element
        div = a[m,m]
        a[m,:] /= div
        b[m] /= div

        # Now subtract from the lower rows
        for i in range(m+1,N):
            mult = a[i,m]
            a[i,:] -= mult*a[m,:]
            b[i] -= mult*b[m]

    return backward_substitution(a,b)

def LU_Doolite(a): #Decomposição LU - Doolite
    n = len(a)
    if np.iscomplexobj(a):
        lower = np.eye(n,dtype=complex)
        upper = np.zeros((n, n),dtype=complex)
    else:
        lower = np.eye(n,dtype=float)
        upper = np.zeros((n, n),dtype=float)

    for i in range(n):
        upper[i, i:] = a[i, i:] - lower[i, :i] @ upper[:i, i:]
        lower[i+1:, i] = (a[i+1:, i] - lower[i+1:, :i] @ upper[:i, i]) / upper[i, i]

    return lower, upper

def LU_Crout(a): #Decomposição LU - Crout
    n = len(a)
    if np.iscomplexobj(a):
        lower = np.eye(n,dtype=complex)
        upper = np.zeros((n, n),dtype=complex)
    else:
        lower = np.eye(n,dtype=float)
        upper = np.zeros((n, n),dtype=float)

    for i in range(n):
        lower[i:, i] = a[i:, i] - lower[i:, :i] @ upper[:i, i]
        upper[i, i+1:] = (a[i, i+1:] - lower[i, :i] @ upper[:i, i+1:]) / lower[i, i]

    return lower, upper

def LU_Chol(a): #Decomposição LU - Cholesky
    n = len(a)
    lower = np.zeros((n, n))

    for i in range(n):
        lower[i, i] = np.sqrt(a[i, i] - lower[i, :i] @ lower[i, :i])
        lower[i+1:, i] = (a[i+1:, i] - lower[i+1:, :i] @ lower[i, :i]) / lower[i, i]

    return lower, lower.T

def solve_LU(a,b): 
    L,U = LU_Doolite(a)
    
    Sol = forward_substitution(L,b)
    
    return backward_substitution(U,Sol)

def LU_decomp_pivot(a):
    n = len(a)
    P = np.eye(n)

    for k in range(n-1):
        # Partial pivoting
        pivot_row = np.argmax(np.abs(a[k:, k])) + k
        P[[k, pivot_row]] = P[[pivot_row, k]]
        
    L,U = LU_Doolite(P@a)

    return P, L, U

def solve_LU_pivot(a,b):
    P,L,U = LU_decomp_pivot(a)
    
    Sol = forward_substitution(L,P@b)
    
    return backward_substitution(U,Sol)

def LU_Crout_Diagonal(a): #Decomposição LU - Crout com diagonal
    n = len(a)
    lower = np.zeros((n, n))
    lower[0, 0] = a[0, 0]
    upper = np.eye(n)

    for i in range(n-1):
        lower[i+1,i] = a[i+1,i]
        upper[i, i+1] = a[i, i+1] / lower[i, i]
        lower[i+1, i+1] = a[i+1, i+1] - lower[i+1, i] * upper[i, i+1]

    return lower, upper

def banded(Aa,va,up,down):

    # Copy the inputs and determine the size of the system
    A = np.copy(Aa)
    v = np.copy(va)
    N = len(v)

    # Gaussian elimination
    for m in range(N):

        # Normalization factor
        div = A[up,m]

        # Update the vector first
        v[m] /= div
        for k in range(1,down+1):
            if m+k<N:
                v[m+k] -= A[up+k,m]*v[m]

        # Now normalize the pivot row of A and subtract from lower ones
        for i in range(up):
            j = m + up - i
            if j<N:
                A[i,j] /= div
                for k in range(1,down+1):
                    A[i+k,j] -= A[up+k,m]*A[i,j]

    # Backsubstitution
    for m in range(N-2,-1,-1):
        for i in range(up):
            j = m + up - i
            if j<N:
                v[m] -= A[i,j]*v[j]

    return v

def QR_Gram(a):
    n = len(a)
    v = np.zeros((n, n))
    Q = np.zeros((n, n))
    threshold = 1e-10

    for i in range(n):
        v[:,i] = a[:,i]
        for k in range(i+1):
            v[:,i] -= np.dot(Q[:,k],a[:,i]) * Q[:,k]
        Q[:,i] = v[:,i] / np.linalg.norm(v[:,i])
    
    Q[np.abs(Q) < threshold] = 0 
    R = Q.T @ a
    R[np.abs(R) < threshold] = 0 

    return Q, R

def QR_Householder(a):
    n = len(a)
    Q = np.eye(n)
    R = a.copy()
    threshold = 1e-10

    for k in range(n):
        Qn = np.eye(n)
        u = R[k:,k] - (-np.sign(R[k,k]) * np.linalg.norm(R[k:,k]) * np.eye(n-k)[0])
        v = u / np.linalg.norm(u)
        Qn[k:,k:] -= 2 * np.outer(v, v.T)
        R = Qn @ R
        Q = Q @ Qn.T
        
    Q[np.abs(Q) < threshold] = 0
    R[np.abs(R) < threshold] = 0
    
    return Q, R

def eigen_QR(a, threshold=1e-9):
    n = len(a)
    A = a.copy()
    Q = np.eye(n)
    for i in range(1000):
        Q, R = QR_Gram(A)
        A = R @ Q
        if np.linalg.norm(np.triu(A, 1)+np.tril(A,-1)) < threshold:
            print(f'Converged in {i} iterations')
            break

    eigenvalues = np.diag(A)
    eigenvectors = Q

    return eigenvalues, eigenvectors

In [ ]:
def relaxacao(f, x0, tol):
    """
    Implementa o método de relaxação para resolver um sistema de equações não lineares.

    Parâmetros:
    f (função->array 1-D): A função que representa o sistema de equações. Do tipo f(x)=x
    x0 (array 1-D): O ponto inicial de aproximação.
    tol (float): A tolerância para a convergência.

    Retorna:
    array: O vetor-solução aproximado.

    """
    erro = tol+1

    for i in range(1000):
        x1 = f(*x0)
        erro = np.linalg.norm(x1-x0)
        x0 = x1

        if erro < tol:
            print(f'Converged in {i} iterations')
            break

    return x0

def sobre_relaxacao(f,x0,tol,w):
    """
    Implementa o método de sobre-relaxação para resolver um sistema de equações lineares.

    Parâmetros:
    f (função->array 1-D): A função que representa o sistema de equações. Do tipo f(x)=x
    x0 (array 1-D): O ponto inicial de aproximação.
    tol (float): A tolerância para a convergência.
    w (float): O fator de relaxação.

    Retorna:
    array: O vetor-solução aproximado.
    """
    erro = tol+1

    for i in range(1000):
        x1 = (1+w)*f(*x0)-w*x0
        erro = np.linalg.norm(x1-x0)
        x0 = x1

        if erro < tol:
            print(f'Converged in {i} iterations')
            break

    return x0

def bissecao(f,a,b,tol):
    """
    Implementa o método da bisseção para encontrar a raiz de uma função no intervalo [a, b].

    Parâmetros:
    f (função): A função para a qual se deseja encontrar a raiz.
    a (float): O limite inferior do intervalo.
    b (float): O limite superior do intervalo.
    tol (float): A tolerância desejada para a raiz.

    Retorna:
    float: A aproximação da raiz da função.

    """
    while (b-a)/2 > tol:
        c = (a+b)/2
        
        if f(a)*f(c) <= 0:
            b = c
        else:
            a = c

    return (a+b)/2

def newton(f,df,x0,tol):
    """
    Implementação do método de Newton-Raphson para encontrar os zeros de uma função.

    Parâmetros:
    - f (função->array 2-D): A função para a qual se deseja encontrar a raiz.
    - df (função->array 2-D): Função que retorna a primeira derivada da função em estudo.
    - x0 (array 2-D): O ponto inicial para iniciar a busca pelo zero. Vetor coluna.
    - tol (float): Tolerância para critério de paragem.

    Retorna:
    - array 1-D: Coordenadas da raiz da função.

    """
    erro = tol+1

    while erro > tol:
        x1 = x0 - np.linalg.inv(df(*x0[:, 0])) @ f(*x0[:, 0])
        erro = np.linalg.norm(x1-x0)
        x0 = x1

    return x0

def secante(f,x0,x1,tol):
    """
    Implementa o método da secante para encontrar uma raiz da função f.

    Parâmetros:
    f (função): A função para a qual se deseja encontrar a raiz.
    x0 (float): O primeiro valor inicial.
    x1 (float): O segundo valor inicial.
    tol (float): A tolerância para o critério de paragem.

    Retorna:
    float: A raiz aproximada da função f.
    """
    erro = tol+1

    while erro > tol:
        x2 = x1 - f(x1)*(x1-x0)/(f(x1)-f(x0))
        erro = np.abs(x2-x1)
        x0 = x1
        x1 = x2
        
    return x2


In [ ]:
def minimo_secaurea(f,x1,x4,tol):
    """
    Função que implementa o método da secção áurea para encontrar o mínimo de uma função unidimensional.

    Parâmetros:
    f (função): Função unidimensional a ser minimizada.
    x1 (float): Limite inferior do intervalo de procura.
    x4 (float): Limite superior do intervalo de procura.
    tol (float): Tolerância para a convergência do método.

    Retorna:
    float: Valor aproximado do mínimo da função.
    """
    phi = (1+np.sqrt(5))/2
    x2 = x4 - (x4-x1)/phi
    x3 = x1 + (x4-x1)/phi
    f2 = f(x2)
    f3 = f(x3)

    while x4-x1 > tol:
        if f2 < f3:
            x4 = x3
            x3 = x2
            x2 = x4 - (x4-x1)/phi
            f3 = f2
            f2 = f(x2)
        else:
            x1 = x2
            x2 = x3
            x3 = x1 + (x4-x1)/phi
            f2 = f3
            f3 = f(x3)
    
    return (x1+x4)/2

def gauss_newton(d1f, d2f, x0, tol):
    """
    Implementação do método de Gauss-Newton para encontrar extremos locais de uma função.

    Parâmetros:
    - d1f (função->array 2-D): Função que retorna o vetor gradiente da função em estudo.
    - d2f (função->array 2-D): Função que retorna a matriz Hessiana da função em estudo.
    - x0 (array 2-D): O ponto inicial para iniciar a busca pelo mínimo. Vetor coluna.
    - tol (float): Tolerância para critério de paragem.

    Retorna:
    - array 1-D: Coordenadas do extremo local.

    """
    erro = tol + 1

    while erro > tol:
        x1 = x0 - np.linalg.inv(d2f(*x0[:, 0])) @ d1f(*x0[:, 0])
        erro = np.linalg.norm(x1 - x0)
        x0 = x1

    return x0[:, 0]

def gradiente(df,x0,tol,gamma):
    """
    Implementa o algoritmo do gradiente descendente para encontrar o mínimo de uma função.

    Parâmetros:
    df (função->array 2-D): A matriz que representa a primeira derivada (o gradiente) da função em estudo.
    x0 (array 2-D): O ponto inicial para iniciar a busca pelo mínimo. Vetor coluna.
    tol (float): A tolerância para o critério de paragem.
    gamma (array 2-D): A matriz do passo para atualização do ponto. Deverá se aproxiamadamente a matriz Hessiana.

    Retorna:
    array 1-D: O ponto que minimiza a função objetivo.

    """
    erro = tol+1

    while erro > tol:
        x1 = x0 - np.linalg.inv(gamma) @ df(*x0[:,0])
        erro = np.linalg.norm(x1-x0)
        x0 = x1

    return x0[:,0]
